# Databricks Community Edition Lab

This is a lab to understand Databricks at the Community Edition level. This notebook is for the complete setup.

In [0]:
%sql
DROP TABLE IF EXISTS orders_raw;
DROP TABLE IF EXISTS orders_cleaned;
DROP TABLE IF EXISTS cn_daily_customer_books;

In [0]:
%sql
-- Creation and usage of schema and volume for data storage
CREATE SCHEMA IF NOT EXISTS STRATA_LAB;
USE SCHEMA STRATA_LAB;
CREATE VOLUME IF NOT EXISTS workspace.strata_lab.entrenamiento;

DROP TABLE IF EXISTS orders_bronze;
DROP TABLE IF EXISTS orders_enriched_temp;
DROP TABLE IF EXISTS orders_updates;
DROP TABLE IF EXISTS orders_raw;
DROP TABLE IF EXISTS orders_cleaned;
DROP TABLE IF EXISTS cn_daily_customer_books;
DROP TABLE IF EXISTS customers_lookup;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS orders_raw;
DROP TABLE IF EXISTS orders_silver;
DROP TABLE IF EXISTS books;
DROP TABLE IF EXISTS customers;
DROP FUNCTION IF EXISTS get_url;
DROP FUNCTION IF EXISTS site_type;

CREATE WIDGET TEXT book_store_path DEFAULT '/Volumes/workspace/strata_lab/entrenamiento/book_store';

Installation of required libraries

In [0]:
# Install required libraries and restart Python environment
%pip install dbldatagen faker
%restart_python

In [0]:
# Retrieve the book store path from the Databricks widget
book_store_path = dbutils.widgets.get("book_store_path")

In [0]:
def load_data(
    row_count=1000,
    save_path="/Volumes/workspace/strata_lab/entrenamiento/book_store/orders/"
):
    """
    Generates synthetic order data and saves it in Parquet format.
    Randomness is introduced so that ~20% of orders contain multiple books.
    """
    from pyspark.sql.functions import (
        col,
        format_string,
        lit,
        struct,
        array,
        to_json,
        unix_timestamp,
        when,
        rand,
        sequence,
        explode,
        element_at,
        shuffle,
        collect_list,
        sum as _sum,
        count,
        first,
        floor,
    )
    from pyspark.sql.types import StructType, StructField, StringType, IntegerType
    books_schema = StructType([
        StructField("book_id_map", StringType(), False),
        StructField("price", IntegerType(), False)
    ])
    from dbldatagen import DataGenerator

    # List of available books and their prices
    books_data = [
        ("B01", 25), ("B02", 28), ("B03", 30), ("B04", 15),
        ("B05", 40), ("B06", 22), ("B07", 18), ("B08", 41),
        ("B09", 50), ("B10", 12), ("B11", 19), ("B12", 29),
    ]
    book_ids = [b[0] for b in books_data]
        
    books_df = spark.createDataFrame(books_data, schema=books_schema)

    # Generate synthetic orders with random customer and order info
    orders_gen = (
        DataGenerator(spark, name="orders_spec", rows=row_count, partitions=4)
        .withColumn("order_id_int", "long", minValue=6341, uniqueValues=row_count)
        .withColumn(
            "order_timestamp_dt",
            "timestamp",
            begin="2022-07-01 00:00:00",
            end="2022-07-31 23:59:59",
            random=True,
        )
        .withColumn("customer_id_int", "int", minValue=1, maxValue=10000, random=True)
        .withColumn("multi_book_rand", "float", minValue=0.0, maxValue=1.0)
    )
    orders_df = orders_gen.build()

    # Assign number of books per order (~20% have multiple books)
    orders_with_book_count = orders_df.withColumn(
        "num_books",
        when(col("multi_book_rand") >= 0.8, (rand() * 3 + 2).cast("int"))
        .otherwise(lit(1)),
    )
    
    # Create a sequence for each book in the order
    line_items_unassigned = orders_with_book_count.withColumn(
        "book_seq_array", sequence(lit(1), col("num_books"))
    ).select("*", explode(col("book_seq_array")).alias("book_seq"))

    # Randomly assign a book_id to each line item
    book_ids_col = array([lit(b) for b in book_ids])
    line_items_with_book = line_items_unassigned.withColumn(
        "book_id", element_at(shuffle(book_ids_col), 1)
    )

    # Assign a random quantity (1-4) for each book
    line_items_with_quantity = line_items_with_book.withColumn(
        "quantity", (floor(rand() * 4) + 1).cast("int")
    )

    # Join with books_df to get the price for each book
    df_with_price = line_items_with_quantity.join(
        books_df,
        line_items_with_quantity["book_id"] == books_df["book_id_map"],
        "left"
    ).withColumn(
        "price", col("price").cast("int")
    )

    # Create a struct for each book item in the order
    df_with_struct = df_with_price.withColumn(
        "book_item",
        struct(
            col("book_id"),
            col("quantity"),
            (col("price") * col("quantity")).alias("subtotal"),
        ),
    )

    # Aggregate line items by order_id to form the final order
    df_aggregated = df_with_struct.groupBy("order_id_int").agg(
        first("order_timestamp_dt").alias("order_timestamp_dt"),
        first("customer_id_int").alias("customer_id_int"),
        _sum(col("price") * col("quantity")).alias("total"),
        _sum("quantity").alias("quantity"),
        collect_list("book_item").alias("books"),
    )

    # Format columns and select final fields for output
    df_final = (
        df_aggregated
        .withColumn("order_id", format_string("%016d", col("order_id_int")))
        .withColumn("order_timestamp", unix_timestamp(col("order_timestamp_dt")))
        .withColumn("customer_id", format_string("C%05d", col("customer_id_int")))
        .select(
            "order_id", "order_timestamp", "customer_id", "quantity", "total", "books"
        )
    )

    # Save the generated orders as Parquet files
    df_final.write.mode("append").format("parquet").save(
        f"{save_path}"
    )

In [0]:
def generate_customer_data(from_scratch=False, nr_new_customers=200):
    """
    Reads the unique customer_id values from the 'orders_bronze' table, generates fake personal data for each one, and saves them in a new location.
    """
    from pyspark.sql.functions import (
        col, lit, struct, to_json, rand, expr,
        monotonically_increasing_id, regexp_extract,
        max as _max, array, element_at, floor
    )
    from dbldatagen import DataGenerator, FakerTextFactory

    # List of email domains for generating fake emails
    email_domains = [
        "example.com", "gmail.com", "hotmail.com", "yahoo.com", "outlook.com",
        "gmil.com", "t-online.de", "university.edu", "protonmail.com", "icloud.com",
        "fastmail.com", "zoho.com", "gmx.net", "web.de", "mail.com"
    ]
    email_domains_col = array([lit(d) for d in email_domains])

    if from_scratch:
        try:
            # Read existing orders to determine the max customer_id
            orders_df = spark.read.parquet("/Volumes/workspace/strata_lab/entrenamiento/book_store/orders/")
            max_id_str = orders_df.select(
                _max(regexp_extract(col("customer_id"), r'C(\d+)', 1).cast("int")).alias("max_id")
            ).collect()[0]["max_id"]
            start_id = max_id_str + 1 if max_id_str is not None else 1
        except Exception as e:
            # If reading fails, start from 1
            start_id = 1

        faker_factory = FakerTextFactory(locale=["en_US"])

        # Generate base customer profile data (without email)
        customer_profile_gen = (
            DataGenerator(
                sparkSession=spark,
                name="customer_profile_spec",
                rows=nr_new_customers,
                partitions=4,
            )
            .withColumn("starting_id", "long", minValue=start_id, maxValue=start_id + nr_new_customers - 1, uniqueValues=nr_new_customers)
            .withColumn("customer_id", expr="format_string('C%05d', starting_id)")
            .withColumn("first_name", text=faker_factory("first_name"))
            .withColumn("last_name", text=faker_factory("last_name"))
            .withColumn("gender", "string", values=["Male", "Female", "Non-binary"])
            .withColumn("street", text=faker_factory("street_address"))
            .withColumn("city", text=faker_factory("city"))
            .withColumn("country", "string", values=["USA"])
            .withColumn("updated", "date", uniqueValues=365, random=True)
        )

        df_generated_profiles = customer_profile_gen.build()

        # Add email column using random domain and names
        df_with_email = (
            df_generated_profiles
            .withColumn("email_domain", element_at(email_domains_col, (floor(rand() * len(email_domains)) + 1).cast("int")))
            .withColumn("email", expr("concat(lower(first_name), '.', lower(last_name), '@', email_domain)"))
        )

        # Create JSON profile structure
        customers = df_with_email.withColumn(
            "profile",
            to_json(
                struct(
                    col("first_name"),
                    col("last_name"),
                    col("gender"),
                    struct(col("street"), col("city"), col("country")).alias("address"),
                )
            )
        ).select("customer_id", "email", "profile", "updated")

        # Save generated customer data as JSON
        customers.repartition(6).write.mode("append").format("json").save(
            "/Volumes/workspace/strata_lab/entrenamiento/book_store/customers-json-new/"
        )

    else:
        try:
            # Read unique customer IDs from orders
            customers_df = spark.read.parquet("/Volumes/workspace/strata_lab/entrenamiento/book_store/orders/").select("customer_id").distinct()
        except Exception as e:
            # Fallback: generate dummy customer IDs if reading fails
            customers_df = spark.range(1, 200).selectExpr("format_string('C%05d', id) as customer_id")

        customer_count = customers_df.count()
        if customer_count == 0:
            return

        faker_factory = FakerTextFactory(locale=["en_US"])

        # Generate base customer profile data (without email)
        customer_profile_gen = (
            DataGenerator(
                sparkSession=spark,
                name="customer_profile_spec",
                baseDF=customers_df,
                rows=customer_count,
                partitions=4,
            )
            .withColumn("first_name", text=faker_factory("first_name"))
            .withColumn("last_name", text=faker_factory("last_name"))
            .withColumn("gender", "string", values=["Male", "Female", "Non-binary"])
            .withColumn("street", text=faker_factory("street_address"))
            .withColumn("city", text=faker_factory("city"))
            .withColumn("country", "string", values=["USA"])
            .withColumn("updated", "date", uniqueValues=365, random=True)
        )

        df_generated_profiles = customer_profile_gen.build()

        # Add join key to preserve customer_id after DataGenerator
        customers_df_with_key = customers_df.withColumn("join_key", monotonically_increasing_id())
        profiles_with_key = df_generated_profiles.withColumn("join_key", monotonically_increasing_id())
        df_merged = customers_df_with_key.join(profiles_with_key, "join_key").drop("join_key")

        # Add email column using random domain and names
        df_with_email = (
            df_merged
            .withColumn("email_domain", element_at(email_domains_col, (floor(rand() * len(email_domains)) + 1).cast("int")))
            .withColumn("email", expr("concat(lower(first_name), '.', lower(last_name), '@', email_domain)"))
        )
        
        # Create JSON profile structure
        customers = df_with_email.withColumn(
            "profile",
            to_json(
                struct(
                    col("first_name"),
                    col("last_name"),
                    col("gender"),
                    struct(col("street"), col("city"), col("country")).alias("address"),
                )
            )
        ).select("customer_id", "email", "profile", "updated")

        # Save generated customer data as JSON
        customers.repartition(6).write.mode("overwrite").format("json").save(
            "/Volumes/workspace/strata_lab/entrenamiento/book_store/customers/"
        )

In [0]:
def clear_workspace():
    """
    Function to delete the generated files and folders
    """
    # Remove the entire training volume directory and its contents
    dbutils.fs.rm("/Volumes/workspace/strata_lab/entrenamiento/", True)

In [0]:
import csv
import os
import random
from faker import Faker

# Output directory for generated CSV files
output_dir = "/Volumes/workspace/strata_lab/entrenamiento/book_store/books-csv"
os.makedirs(output_dir, exist_ok=True)

fake = Faker()
def generate_books():
    # List of possible book categories
    categories = [
    "Computer Science", "Mathematics", "Physics",
    "Philosophy", "Literature", "History", "Art"
    ]
    books = []
    # Generate 12 fake books with random data
    for i in range(1, 13):
        book = {
            "book_id": f"B{i:02d}",
            "title": fake.sentence(nb_words=4).replace(",", ""), 
            "author": fake.name(),
            "category": random.choice(categories),            
            "price": fake.random_int(min=10, max=100)
        }
        books.append(book)

    # Split books into 3 CSV files, 4 books each
    for file_idx in range(3):
        start = file_idx * 4
        end = start + 4
        chunk = books[start:end]

        file_path = os.path.join(output_dir, f"books_part_{file_idx+1}.csv")

        # Write chunk to CSV file with semicolon delimiter
        with open(file_path, mode="w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=["book_id", "title", "author", "category", "price"], delimiter=";")
            writer.writeheader()
            writer.writerows(chunk)

    # Also generate table for SQL querying
    from pyspark.sql.functions import col
    spark.sql("DROP TABLE IF EXISTS books")  # Drop table if it exists

    # Read all generated CSV files into a Spark DataFrame
    books_df = spark.read.option("header", "true") \
        .option("delimiter", ";") \
    .csv(f"{dbutils.widgets.get('book_store_path')}/books-csv")

    # Cast price column to double type
    books_df = books_df.withColumn("price", col("price").cast("double"))

    # Save DataFrame as a Spark SQL table
    books_df.write.saveAsTable("books")


In [0]:
import csv
import os
def new_books_csv():

    output_dir = "/Volumes/workspace/strata_lab/entrenamiento/book_store/books-csv-new"
    os.makedirs(output_dir, exist_ok=True)

    # Define the header and the rows of data
    header = ['book_id', 'title', 'author', 'category', 'price']
    data = [
        ['B14', 'Data Communications and Networking', 'Behrouz A. Forouzan', 'Computer Science', 34],
        ['B15', 'Inside the Java Virtual Machine', 'Bill Venners', 'Computer Science', 41],
        ['B13', 'Linux pocket guide', 'Daniel J. Barrett', 'Computer Science', 26],
        ['B16', 'Green for Life', 'Victoria Boutenko', 'Food', 18],
        ['B17', 'Cooking with Love', 'Carla Hall', 'Food', 23]
    ]

    # Specify the filename
    file_path = os.path.join(output_dir, "books.csv")
    # Write the data to the CSV file
    with open(file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Write the header
        writer.writerow(header)
        
        # Write the data rows
        writer.writerows(data)



In [0]:
def add_books_cdc():
    import json

    # Creamos las carpetas a utilizar dentro del volumen de entrenamiento
    # Buscamos las carpetas en el volumen
    dbutils.fs.mkdirs("/Volumes/workspace/strata_lab/entrenamiento/books-cdc/")

    # Data structured as a list of dictionaries, representing the rows from the table.
    # SQL NULL values are represented as None in Python, which translates to null in JSON.
    book_data = [
        {
            "id": 1,
            "author": "Addy Osmani",
            "book_id": "B02",
            "category": "Computer Science",
            "price": 40,
            "row_status": "UPDATE",
            "row_time": "2022-11-05T17:50:23.601Z",
            "title": "Learning JavaScript Design Patterns"
        },
        {
            "id": 2,
            "author": "Tariq Rashid",
            "book_id": "B03",
            "category": "Computer Science",
            "price": 30,
            "row_status": "UPDATE",
            "row_time": "2022-11-05T10:11:33.507Z",
            "title": "Make Your Own Neural Network"
        },
        {
            "id": 3,
            "author": "null",
            "book_id": "B01",
            "category": "null",
            "price": "null",
            "row_status": "DELETE",
            "row_time": "2022-11-05T17:50:23.601Z",
            "title": "null"
        },
        {
            "id": 4,
            "author": "Mark W. Spong",
            "book_id": "B04",
            "category": "Computer Science",
            "price": 20,
            "row_status": "INSERT",
            "row_time": "2022-11-05T11:12:05.419Z",
            "title": "Robot Dynamics and Control"
        },
        {
            "id": 5,
            "author": "Luciano Ramalho",
            "book_id": "B05",
            "category": "Computer Science",
            "price": 47,
            "row_status": "INSERT",
            "row_time": "2022-11-05T11:12:05.419Z",
            "title": "Fluent Python"
        },
        {
            "id": 6,
            "author": "François Chollet",
            "book_id": "B06",
            "category": "Computer Science",
            "price": 22,
            "row_status": "INSERT",
            "row_time": "2022-11-05T11:12:05.419Z",
            "title": "Deep Learning with Python"
        }
    ]

    # Define the output filename
    output_filename = 'book_data.json'

    # Write the data to the JSON file with indentation for readability
    try:
        with open("/Volumes/workspace/strata_lab/entrenamiento/books-cdc/02.json", 'w') as json_file:
            for book in book_data:
                json_file.write(f"{book}\n")
    except IOError as e:
        print(f"Error writing to file: {e}")


In [0]:
# Primera carga inical
clear_workspace()
load_data()
load_data(700, save_path=book_store_path+'/orders_new')
generate_customer_data()
generate_customer_data(from_scratch=True)
generate_books()
new_books_csv()
add_books_cdc()
print("Workspace fully generated")